## Land Mark Group Final Solution

In [1]:
# %load required.py
import pandas as pd
import numpy as np
import sklearn
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings(action='ignore')

### Test Data

In [2]:
test_data=pd.read_excel("../Test_Set.xlsx")

In [3]:
test_data.head()

,Customer_ID,Store_Code
0,1800000006365760,60298
1,1800000006365760,60299
2,1800000006365760,60300
3,1800000006365760,60301
4,1800000006365820,60298


### Customer Transaction

In [4]:
customer_transactions=pd.read_excel('../Customer_Transaction.xlsx')

In [5]:
customer_transactions.head()

,Territory,Business,Year,Week,Store_Code,City_Name,Store_Type,Transaction_Type,Return_Reason,Customer_ID,Invoices,Item_Count,Revenue,Discount,Units_Sold
0,United Arab Emirates,Max,2010,0,60065,Dubai,Stand alone,Return,Size Problem,1800000058056860,1,3,-169.0,0.0,-3
1,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000020917140,1,1,0.0,0.0,0
2,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000021265010,1,1,0.0,0.0,0
3,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000025693950,1,4,0.0,0.0,0
4,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000044104620,1,1,0.0,0.0,0


In [6]:
t_count=customer_transactions[['Customer_ID','Store_Code','Year']].groupby(['Customer_ID','Store_Code'])['Year'].agg({"Transaction Count":len}).reset_index()

### No. of transactions a customer has done in any store

In [7]:
customer_store_transactions=t_count.sort_values(['Customer_ID','Store_Code']).pivot_table(values='Transaction Count', index='Customer_ID', columns='Store_Code').reset_index()
customer_store_transactions.fillna(0,inplace=True)
# customer_store_transactions.to_csv("./customer_store_transaction.csv",index=False)

### Creating a flag whether a customer had made any transaction for purticular store or not ?

In [8]:
for col in customer_store_transactions.columns[1:]:
    customer_store_transactions[col]=customer_store_transactions[col].apply(lambda x:1 if x>0 else 0)
#     customer_store_transactions[col]=customer_store_transactions[col].apply(lambda x:1 if x>1 else 0)

In [9]:
customer_store_transactions.head()

Store_Code,Customer_ID,60053,60054,60055,60057,60058,60059,60060,60061,60062,...,60076,60077,60079,60084,60086,60219,60233,60241,60250,60275
0,1800000006365760,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1800000006365820,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1800000006366060,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,1800000006366230,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1800000006366570,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# customer_store_transactions.to_csv("./customer_store_transaction_flags.csv",index=False)

### Customer Demographics

In [10]:
customer_demographics=pd.read_excel("../Customer_Demographics.xlsx")

In [11]:
customer_demographics.columns

Index(['Customer_ID', 'Territory', 'Nationality', 'Income_Range', 'Job_Type',
       'Marital_Status', 'Gender', 'State', 'Language', 'Loyalty_Status',
       'Birth_date', 'Age', 'Points', 'First_txn_dt', 'Last_accr_txn_dt',
       'Last_rdm_txn_dt'],
      dtype='object')

In [12]:
# print("Different Demographics cutomer belong to :",customer_demographics.Territory.unique())
# print("Different Nationality cutomer belong to : \n",customer_demographics.Nationality.value_counts().head(10)*100/customer_demographics.shape[0])
# print("Different Income_Range cutomer belong to : \n",customer_demographics.Income_Range.value_counts())
# print("Different Job_Type cutomer belong to : \n",customer_demographics.Job_Type.value_counts())
# print("Different Marital_Status cutomer belong to : \n",customer_demographics.Marital_Status.value_counts())
# print("Different Gender cutomer belong to : \n",customer_demographics.Gender.value_counts())
print("Different State cutomer belong to : \n",customer_demographics.State.value_counts())

Different State cutomer belong to : 
 Abu Dhabi         41715
Dubai             23189
Others            11619
Sharjah           10919
Unspecified        4804
Ras Al Khaimah     2840
Ajman              2593
Fujairah           2074
Umm Al Quwain       243
Bahrain               1
Qatar                 1
Muscat                1
Kuwait City           1
Name: State, dtype: int64


In [13]:
customer_demographics.head()

,Customer_ID,Territory,Nationality,Income_Range,Job_Type,Marital_Status,Gender,State,Language,Loyalty_Status,Birth_date,Age,Points,First_txn_dt,Last_accr_txn_dt,Last_rdm_txn_dt
0,1800000006365760,United Arab Emirates,INDIA,Below 5000,Services,Married,M,Abu Dhabi,English,Gold,01JUN1959:00:00:00,59.0,814.0,02NOV2010:00:00:00,27SEP2018:21:45:25,11MAR2018:19:11:57
1,1800000006365820,United Arab Emirates,UNITED ARAB EMIRATES,Below 5000,Services,Married,F,Abu Dhabi,Arabic,Silver,01JAN1900:00:00:00,NaN,212.0,06DEC2010:00:00:00,16AUG2016:21:08:28,16MAR2015:13:39:12
2,1800000006366060,United Arab Emirates,SRI LANKA,Below 5000,Services,Married,M,Abu Dhabi,English,Silver,07JUN1959:00:00:00,59.0,186.0,17FEB2011:00:00:00,31AUG2018:20:52:18,30JAN2017:19:19:19
3,1800000006366230,United Arab Emirates,JORDAN,Below 5000,Services,Married,F,Abu Dhabi,Arabic,Silver,21JAN1962:00:00:00,57.0,126.0,30OCT2010:00:00:00,30JUL2018:20:30:05,20JUN2017:17:57:27
4,1800000006366570,United Arab Emirates,OCCUPIED PALESTINIAN TERRITORY,Below 5000,Services,Married,M,Abu Dhabi,Arabic,Silver,01JAN1900:00:00:00,NaN,197.0,10NOV2010:00:00:00,03SEP2018:22:10:56,17JUN2017:23:08:28


In [14]:
customer_demographics[['Gender','Loyalty_Status','State','Income_Range']].describe()

,Gender,Loyalty_Status,State,Income_Range
count,100000,100000,100000,100000
unique,3,2,13,7
top,M,Silver,Abu Dhabi,Unspecified
freq,52524,92196,41715,92853


In [15]:
import dateutil
customer_demographics['lt_accr_date']=customer_demographics.Last_accr_txn_dt.apply(lambda x:None if ((x=='nan') or (x is np.nan) or (x==np.nan)) else dateutil.parser.datetime.datetime.strptime(x,"%d%b%Y:%H:%M:%S"))
customer_demographics['lt_rdm_date']=customer_demographics.Last_rdm_txn_dt.apply(lambda x:None if ((x=='nan') or (x is np.nan) or (x==np.nan)) else dateutil.parser.datetime.datetime.strptime(x,"%d%b%Y:%H:%M:%S"))
customer_demographics['ft_txn_date']=customer_demographics.First_txn_dt.apply(lambda x:None if ((x=='nan') or (x is np.nan) or (x==np.nan)) else dateutil.parser.datetime.datetime.strptime(x,"%d%b%Y:%H:%M:%S"))

In [16]:
customer_demographics['ds_ft_purchase']=(customer_demographics['lt_accr_date']-customer_demographics['ft_txn_date'])
customer_demographics['lt_date']=customer_demographics[['lt_accr_date','lt_rdm_date']].apply(max,axis=1)

In [17]:
customer_demographics['lt_month']=customer_demographics['lt_date'].apply(lambda x:x.month)
customer_demographics['lt_year']=customer_demographics['lt_date'].apply(lambda x:x.year)

### Store Dataset

In [18]:
store_master=pd.read_excel("../Store_Master.xlsx")

In [19]:
store_master.head()

,Territory,Business,Region,Region_Code,Store_Code,Store_Name,Mall_Name,Store_Format,Sales_Per_Day,Store_Launch_Date,Train_Test_Store,Geo_Field,Store_Size_Sq_Ft,Customer_Count,Total_Revenue
0,United Arab Emirates,Max,MX - Abu Dhabi,602,60077,Max Zayed City Mall,City Mall - Abu Dhabi-Uae,NaN,544.074849,2011-11-02,Train,"25.618167, 53.441646",4852.220341,22237.0,1.131676e+07
1,United Arab Emirates,Max,MX - Sharjah,606,60073,Max Safeer Mall,Safeer Mall Sharjah-Uae,NaN,1251.656549,2009-08-21,Train,"25.309997, 55.378550",13789.949661,88372.0,2.603446e+07
2,United Arab Emirates,Max,MX - Dubai,601,60065,Max Gardens,Ibn Battuta Mall-Uae,NaN,3319.885702,2005-04-15,Train,"25.044684, 55.120275",15609.687991,188836.0,6.905362e+07
3,United Arab Emirates,Max,MX - Dubai,601,60219,MX -Dubai Festival City,Dubai Festival City-Uae,Stand Alone Stores,1211.477261,2014-06-12,Train,"25.218486, 55.359313",6845.497058,96733.0,2.519873e+07
4,United Arab Emirates,Max,MX - Ras Al Khaimah,605,60071,Max Rak,STANDALONE,NaN,2221.872874,2006-03-20,Train,NaN,12385.047461,70456.0,4.621496e+07


In [20]:
store_master[store_master.Train_Test_Store=='Test']

,Territory,Business,Region,Region_Code,Store_Code,Store_Name,Mall_Name,Store_Format,Sales_Per_Day,Store_Launch_Date,Train_Test_Store,Geo_Field,Store_Size_Sq_Ft,Customer_Count,Total_Revenue
31,United Arab Emirates,Max,MX - Abu Dhabi,602,60301,MAX- Abudhabi Al Wahda,AL Wahda,Mall Stores,10861.009520,2018-01-30,Test,"24.470587, 54.372679",23288.338616,NaN,7472374.550
32,United Arab Emirates,Max,MX - Sharjah,606,60298,MAX - Matejer Al Juraina,Matejer Al Juraina,Mall Stores,4099.543401,2018-01-07,Test,"25.296484, 55.489202",8391.589347,NaN,2820485.860
33,United Arab Emirates,Max,MX - Abu Dhabi,602,60300,MAX-WTC,WTC,Mall Stores,7036.233391,2018-02-01,Test,"24.487658, 54.357529",6604.762130,NaN,2701913.622
34,United Arab Emirates,Max,MX - Abu Dhabi,602,60299,MAX-Alain Mall,Alain Mall,Mall Stores,5157.825699,2018-02-01,Test,"24.222016, 55.781629",3266.388547,NaN,479677.790


In [21]:
store_master[store_master.Geo_Field.isnull()&store_master.Region_Code.isin((602,606))]

,Territory,Business,Region,Region_Code,Store_Code,Store_Name,Mall_Name,Store_Format,Sales_Per_Day,Store_Launch_Date,Train_Test_Store,Geo_Field,Store_Size_Sq_Ft,Customer_Count,Total_Revenue
6,United Arab Emirates,Max,MX - Sharjah,606,60075,Max Sharjah Qassimia,STANDALONE,NaN,2268.223683,2011-02-11,Train,NaN,9582.354867,113795.0,4.717905e+07
15,United Arab Emirates,Max,MX - Sharjah,606,60074,Max Sharjah,STANDALONE,NaN,2631.139338,2004-07-21,Train,NaN,3467.799125,137253.0,5.472770e+07
28,United Arab Emirates,Max,MX - Abu Dhabi,602,60053,Max Abudhabi,STANDALONE,NaN,3094.424495,2004-05-16,Train,NaN,19200.122169,160218.0,6.436403e+07


In [ ]:
# store_master[(store_master.Region_Code==602)&(store_master.Train_Test_Store!='Test')]

## Solution 1 : Customers of stores in same region and visited to newely opened stores are most likely to visit the newely opened store

In [22]:
from itertools import combinations_with_replacement,combinations
import geopy.distance

comb=list(combinations(list(store_master[(store_master['Region_Code'].isin([602,606]))&\
                                         (~store_master['Geo_Field'].isnull())]['Geo_Field'].values),2))

print("$"*30," Total Number of Combination : ",len(comb),"$"*30)
print()
print()

store_distance=[]
for i in comb[:]:
    coords_1,coords_2=tuple(map(float,i[0].split(","))),tuple(map(float,i[1].split(",")))
    store_distance.append({"source":i[0],"destination":i[1],"distance":geopy.distance.vincenty(coords_1, coords_2).km})

print("$"*35," Distance Dataframe ","$"*35)
print()
print()
   
distance_df=pd.DataFrame.from_records(store_distance)
print(distance_df.head(10))


$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$  Total Number of Combination :  105 $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$


$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$  Distance Dataframe  $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$


            destination    distance                source
0  25.309997, 55.378550  197.756704  25.618167, 53.441646
1  24.476186, 54.321439  154.545281  25.618167, 53.441646
2  24.312255, 54.619651  187.283174  25.618167, 53.441646
3  25.325119, 55.393559  198.951334  25.618167, 53.441646
4  24.434721, 54.412599  163.671819  25.618167, 53.441646
5  24.321044, 54.480138  177.878220  25.618167, 53.441646
6  24.322808, 54.480221  177.724939  25.618167, 53.441646
7  24.170905, 55.808654  287.904123  25.618167, 53.441646
8  24.333514, 54.523955  179.420909  25.618167, 53.441646
9  24.503005, 54.382945  155.820654  25.618167, 53.441646


In [23]:
# print(store_master[(store_master['Region_Code'].isin([602,606]))&\
#              (~store_master['Geo_Field'].isnull())][['Store_Code','Geo_Field']].Geo_Field.unique())

# len(set(distance_df.destination.unique()).union(set(distance_df.source.unique())))

In [24]:
distance_df_map=pd.merge(pd.merge(distance_df,store_master[(store_master['Region_Code'].isin([602,606]))&(~store_master['Geo_Field'].isnull())][['Store_Code','Geo_Field']],left_on='source',right_on='Geo_Field',how='left'),store_master[(store_master['Region_Code'].isin([602,606]))&(~store_master['Geo_Field'].isnull())][['Store_Code','Geo_Field']],how='left',left_on='destination',right_on='Geo_Field')[['Store_Code_x','Store_Code_y','distance']]
distance_df_map.columns=['Source_Store_Code','Destination_Sorce_Code','Distance']

distance_df_final=distance_df_map[distance_df_map.Source_Store_Code!=distance_df_map.Destination_Sorce_Code]
distance_df_final=distance_df_map.copy()
del distance_df_map,distance_df

In [25]:
distance_df_final.head()

,Source_Store_Code,Destination_Sorce_Code,Distance
0,60077,60073,197.756704
1,60077,60067,154.545281
2,60077,60058,187.283174
3,60077,60076,198.951334
4,60077,60069,163.671819


#### Develop hypothesis based on store proximity 

In [26]:
distance_df_final[distance_df_final.Source_Store_Code.isin([60298])|distance_df_final.Destination_Sorce_Code.isin([60298])].sort_values("Distance").head()

# hypo-thesis based on store distance 
# customer from store 60076 and 60073 might visit 60298
# 60057,60058 would never visit 

customer_60298=customer_store_transactions[['Customer_ID',60076,60075]] # 60073,60074,
customer_60298['pred']=customer_60298[[60076,60075]].apply(max,axis=1) #60073,60074,
customer_60298['Store_Code']=60298

In [27]:
# distance_df_final[distance_df_final.Source_Store_Code.isin([60298])|distance_df_final.Destination_Sorce_Code.isin([60298])].sort_values("Distance")

In [28]:
distance_df_final[distance_df_final.Source_Store_Code.isin([60299])|distance_df_final.Destination_Sorce_Code.isin([60299])].sort_values("Distance").head()

# hypo-thesis based on store distance 
# customer from store 60057 might visit 60299
# 60058,60298,60073,60054 would never visit  store 60299

customer_60299=customer_store_transactions[['Customer_ID',60057]]
customer_60299['pred']=customer_60299[[60057]].apply(max,axis=1)
customer_60299['Store_Code']=60299

In [29]:
distance_df_final[distance_df_final.Source_Store_Code.isin([60300])|distance_df_final.Destination_Sorce_Code.isin([60300])].sort_values("Distance").head(10)

# hypo-thesis based on store distance 
# customer from store 60275,60067,60069 might visit 60300
# 60233 would never visit  store 60300

customer_60300=customer_store_transactions[['Customer_ID',60275,60067,60053]]
customer_60300['pred']=customer_60300[[60275,60067,60053]].apply(max,axis=1)
customer_60300['Store_Code']=60300

In [30]:
distance_df_final[distance_df_final.Source_Store_Code.isin([60301])|distance_df_final.Destination_Sorce_Code.isin([60301])].sort_values("Distance").head(10)

# hypo-thesis based on store distance 
# customer from store 60275,60067,60069,60233,60079,60054,60053,60077 might visit 60301
# 60058 would never visit  store 60301

customer_60301=customer_store_transactions[['Customer_ID',60275,60067,60069,60233,60079,60054,60053,60077]] #60054,60058
customer_60301['pred']=customer_60301[[60275,60067,60069,60233,60079,60054,60053,60077]].apply(max,axis=1)
customer_60301['Store_Code']=60301
# customer_60301['proximity_count']=customer_60301[[60275,60067,60069,60233,60079,60054,60053,60077]].apply(sum,axis=1)
# customer_60301['pred']=customer_60301['proximity_count'].apply(lambda x:1 if x==1 else 0)

In [31]:
result=pd.concat([customer_60298[['Customer_ID','pred','Store_Code']],customer_60299[['Customer_ID','pred','Store_Code']],\
           customer_60300[['Customer_ID','pred','Store_Code']],customer_60301[['Customer_ID','pred','Store_Code']]],axis=0)
result.rename(columns={'pred':'Prediction'},inplace=True)
result[['Customer_ID','Store_Code','Prediction']]['Store_Code'].value_counts()

60298    100000
60299    100000
60300    100000
60301    100000
Name: Store_Code, dtype: int64

In [32]:
pd.merge(test_data,result[['Customer_ID','Store_Code','Prediction']]).to_csv("./distance.csv",index=False)

### Tableau Geo-Plotting for Max stores 

<a href="https://public.tableau.com/profile/sajan5856#!/vizhome/MaxStoresGeoPlotting/RegionWiseStoreGeoPlotting?publish=yes"> Please visit the link to view the report </a>

<img src="./Plot.jpg">

### Customers who had made purchase in last 4 months or after may 2018 is most likely to visit the store in the same region

In [33]:
def receny_pred(row):
    if((row['lt_year']==2018) and (row['lt_month']>5)): #5
        return 1
    else:
        return 0

In [34]:
customer_demographics['pred']=customer_demographics.apply(receny_pred,axis=1)

In [35]:
df=pd.merge(result[['Customer_ID','Store_Code','Prediction']],customer_demographics[['Customer_ID','pred']],left_on='Customer_ID',right_on='Customer_ID',how='left')

In [36]:
def recency_twik(row):
    if(row['Prediction']):
        if(row['pred']):
            return 1
        elif(~row['pred']):
            return 0
    else:
        return 0


In [37]:
# df[df.pred.isnull()]
print(df.Prediction.value_counts())
print(df.pred.value_counts())

0    306897
1     93103
Name: Prediction, dtype: int64
1    269268
0    130732
Name: pred, dtype: int64


In [38]:
df['prediction']=df.apply(recency_twik,axis=1)

In [39]:
df['prediction'].value_counts()

0    325873
1     74127
Name: prediction, dtype: int64

In [40]:
pd.merge(test_data,df[['Customer_ID','Store_Code','prediction']]).rename(columns={'prediction':'Prediction'}).to_csv("./proximity_hypothesis_submission.csv",index=False)